In [17]:
import collections
import helper
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

Using TensorFlow backend.


In [0]:
%load helpers.py
%load project_tests.py

In [2]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
!/opt/bin/nvidia-smi

Mon Feb  4 10:16:00 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.44                 Driver Version: 396.44                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    78W / 149W |    116MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [4]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 7455185437140197737, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 6132996443958781302
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 14999909693593503829
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11281553818
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 14001893657703756815
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [7]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [9]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 授权登录，仅第一次的时候会鉴权
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
file_list = drive.ListFile({'q': "trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s, mimeType: %s' % (file1['title'], file1['id'], file1["mimeType"]))

    100% |████████████████████████████████| 993kB 18.8MB/s 
title: nmt.ipynb, id: 1Q0xNXu_n6wQoRu6YgHUCROExvtR-ksie, mimeType: application/vnd.google.colaboratory
title: small_vocab_fr, id: 14-DuTtlb3S19rdcJOweAryKzrFYivByz, mimeType: application/octet-stream
title: small_vocab_en, id: 1twaElSkNcEtwvi2yvqWUeHgINhESk-6L, mimeType: application/octet-stream
title: small_vocab_fr, id: 1Ri2cW3ttTFUM2bXN0FN_ZCz5luSvPTt3, mimeType: application/octet-stream
title: small_vocab_en, id: 1x_G-m6yNCPnlpiEeWF7EvNzz8qraQJhZ, mimeType: application/octet-stream
title: data, id: 1umTdiI_1To6vMR0EqrEt3fVeM-Un6ZHo, mimeType: application/vnd.google-apps.folder
title: RNN.ipynb, id: 178ogaT-9YHSHIN7V7xlQKuUYyWWcrb5p, mimeType: application/vnd.google.colaboratory
title: vac.txt, id: 1tTlnw5xs89cRLdv_HkEmwVfhnep9ZSll, mimeType: text/plain
title: Colab Notebooks, id: 1VlNKj2c4SAh4XxDjXuVzjPdgWtu4AENs, mimeType: application/vnd.google-apps.folder
title: Technology, Innovation and Entrepreneurial Impact.xlsx, id

In [0]:
def load_data(path):
    """
    Load dataset
    """
    input_file = os.path.join(path)
    with open(input_file, "r") as f:
        data = f.read()

    return data.split('\n')

In [13]:
import os
os.chdir("drive/Colab Notebooks")
english_sentences = load_data('small_vocab_en')
french_sentences = load_data('small_vocab_fr')
print('Dataset Loaded')

Dataset Loaded


In [14]:
for sample_i in range(2):
  print('small_vocab_en Line {}: {}'.format(sample_i + 1, english_sentences[sample_i]))
  print('small_vocab_fr Line {}: {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1: new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1: new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2: the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2: les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


In [21]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])
print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


In [18]:
def tokenize(x):
  x_tk = Tokenizer(char_level = False)
  x_tk.fit_on_texts(x)
  return x_tk.texts_to_sequences(x), x_tk
text_sentences = [
 'The quick brown fox jumps over the lazy dog .',
 'By Jove , my quick study of lexicography won a prize .',
 'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
  print('Sequence {} in x'.format(sample_i + 1))
  print(' Input: {}'.format(sent))
  print(' Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
 Input: The quick brown fox jumps over the lazy dog .
 Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
 Input: By Jove , my quick study of lexicography won a prize .
 Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
 Input: This is a short sentence .
 Output: [18, 19, 3, 20, 21]


In [22]:
def pad(x, length=None):
  if length is None:
    length = max([len(sentence) for sentence in x])
  return pad_sequences(x, maxlen = length, padding = 'post')
tests.test_pad(pad)
# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
  print('Sequence {} in x'.format(sample_i + 1))
  print(' Input: {}'.format(np.array(token_sent)))
  print(' Output: {}'.format(pad_sent))

NameError: ignored

In [26]:
def preprocess(x, y):
  preprocess_x, x_tk = tokenize(x)
  preprocess_y, y_tk = tokenize(y)
  preprocess_x = pad(preprocess_x)
  preprocess_y = pad(preprocess_y)
# Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
  preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)
  return preprocess_x, preprocess_y, x_tk, y_tk
preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
preprocess(english_sentences, french_sentences)

max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)
print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


In [27]:
def logits_to_text(logits, tokenizer):
  index_to_words = {id: word for word, id in tokenizer.word_index.items()}
  index_to_words[0] = '<PAD>'
  return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])
print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [33]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):

  learning_rate = 1e-3
  model = Sequential()
  model.add(GRU(128, input_shape = input_shape[1:], return_sequences = False))
  model.add(RepeatVector(output_sequence_length))
  model.add(GRU(128, return_sequences = True))
  model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
 
  model.compile(loss = sparse_categorical_crossentropy, 
  optimizer = Adam(learning_rate), 
  metrics = ['accuracy'])
  return model
tests.test_encdec_model(encdec_model)
tmp_x = pad(preproc_english_sentences)
tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[1], 1))
encodeco_model = encdec_model(
 tmp_x.shape,
 preproc_french_sentences.shape[1],
 len(english_tokenizer.word_index)+1,
 len(french_tokenizer.word_index)+1)
encodeco_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)
print(logits_to_text(encodeco_model.predict(tmp_x[:1])[0], french_tokenizer))

NameError: ignored

In [0]:
def test_model_final(model_final):
    input_shape = (137861, 15)
    output_sequence_length = 21
    english_vocab_size = 199
    french_vocab_size = 344

    model = model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size)
    _test_model(model, input_shape, output_sequence_length, french_vocab_size)


In [37]:
def final_predictions(x, y, x_tk, y_tk):
  tmp_X = pad(preproc_english_sentences)
  model = encdec_model(tmp_X.shape,
  preproc_french_sentences.shape[1],
  len(english_tokenizer.word_index)+1,
  len(french_tokenizer.word_index)+1)
 
  model.fit(tmp_X, preproc_french_sentences, batch_size = 1024, epochs = 17, validation_split = 0.2)
 
  y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
  y_id_to_word[0] = '<PAD>'
  sentence = 'he saw a old yellow truck'
  sentence = [x_tk.word_index[word] for word in sentence.split()]
  sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
  sentences = np.array([sentence[0], x[0]])
  predictions = model.predict(sentences, len(sentences))
  print('Sample 1:')
  print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
  print('Il a vu un vieux camion jaune')
  print('Sample 2:')
  print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
  print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))
  
final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

NameError: ignored